In [1]:
pip install pyspark duckdb pandas ftfy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
import glob
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, to_date, when, lit, input_file_name
import duckdb

# --- Configuration Paths ---
# Use raw string (r"...") to handle Windows backslashes
BASE_PATH = r"C:\Users\jinna\Downloads\inside-airbnb-bangkok-data"

# Path to the folder containing date-partitioned raw data
RAW_BASE_PATH = os.path.join(BASE_PATH, "raw_data")

# Output paths
PROCESSED_PATH = os.path.join(BASE_PATH, "processed_data")
DB_PATH = os.path.join(BASE_PATH, "database", "airbnb_dw.duckdb")

# --- Initialize Spark Session ---
spark = SparkSession.builder \
    .appName("AirbnbLocalBatch") \
    .master("local[*]") \
    .config("spark.sql.legacy.timeParserPolicy", "CORRECTED") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print("Spark Session Created Successfully!")
print(f"Base Data Path: {RAW_BASE_PATH}")

Spark Session Created Successfully!
Base Data Path: C:\Users\jinna\Downloads\inside-airbnb-bangkok-data\raw_data


In [3]:
# ETL for Listings and Hosts
# --- 1. Extract ---
# Search for all listing files in subfolders
search_pattern = os.path.join(RAW_BASE_PATH, "*", "listings*.csv.gz")
print(f"Searching for: {search_pattern}")

target_files = glob.glob(search_pattern)
print(f"Found {len(target_files)} files.")

if not target_files:
    raise Exception("No listings files found. Please check the directory structure.")

# Read all found files
listings_df = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiLine", "true") \
    .option("escape", "\"") \
    .csv(target_files)

print(f"Total Raw Rows: {listings_df.count()}")

# --- 2. Transform (Host Dimension) ---
df_hosts = listings_df.select(
    col("host_id").cast("long"),
    col("host_name"),
    to_date(col("host_since"), "yyyy-MM-dd").alias("host_since_date"),
    col("host_location"),
    col("host_response_time"),
    
    # Clean Host Response Rate: Handle 'N/A', remove '%', cast to float then decimal
    ((regexp_replace(
        when(col("host_response_rate") == "N/A", None).otherwise(col("host_response_rate")), 
        "%", ""
    ).cast("float") / 100).cast("decimal(5,2)")).alias("host_response_rate"),
    
    # Clean Host Acceptance Rate
    ((regexp_replace(
        when(col("host_acceptance_rate") == "N/A", None).otherwise(col("host_acceptance_rate")), 
        "%", ""
    ).cast("float") / 100).cast("decimal(5,2)")).alias("host_acceptance_rate"),

    # Clean Boolean (t/f -> True/False)
    (when(col("host_is_superhost") == 't', True)
     .when(col("host_is_superhost") == 'f', False)
     .otherwise(None)).alias("is_superhost"),
     
    col("host_listings_count").cast("int"),
    
    (when(col("host_identity_verified") == 't', True)
     .when(col("host_identity_verified") == 'f', False)
     .otherwise(None)).alias("is_identity_verified")
).dropDuplicates(['host_id'])

# --- 3. Transform (Listing Dimension) ---
df_listings = listings_df.select(
    col("id").cast("long").alias("listing_id"),
    col("name").alias("listing_name"),
    col("host_id").cast("long"),
    col("neighbourhood_cleansed").alias("neighbourhood"),
    col("latitude").cast("decimal(10,7)"),
    col("longitude").cast("decimal(10,7)"),
    col("property_type"),
    col("room_type"),
    col("accommodates").cast("int"),
    col("bedrooms").cast("int"),
    col("beds").cast("int"),
    
    # Clean Price: Remove $ and ,
    (regexp_replace(col("price"), "[$,]", "").cast("decimal(10, 2)")).alias("default_price"),
    
    col("number_of_reviews").cast("int"),
    col("review_scores_rating").cast("decimal(3,2)")
).dropDuplicates(['listing_id'])

# --- 4. Load (Write to Parquet) ---
os.makedirs(PROCESSED_PATH, exist_ok=True)

print("Writing Hosts data...")
df_hosts.write.mode("overwrite").parquet(os.path.join(PROCESSED_PATH, "hosts"))

print("Writing Listings data...")
df_listings.write.mode("overwrite").parquet(os.path.join(PROCESSED_PATH, "listings"))

print("Listings & Hosts Processed Successfully!")

Searching for: C:\Users\jinna\Downloads\inside-airbnb-bangkok-data\raw_data\*\listings*.csv.gz
Found 4 files.
Total Raw Rows: 110159
Writing Hosts data...
Writing Listings data...
Listings & Hosts Processed Successfully!


In [4]:
# ETL for Calendar
# Extract 
search_pattern_cal = os.path.join(RAW_BASE_PATH, "*", "calendar*.csv.gz")
print(f"Searching for: {search_pattern_cal}")

target_files_cal = glob.glob(search_pattern_cal)
print(f"Found {len(target_files_cal)} files.")

if not target_files_cal:
    raise Exception("No calendar files found.")

calendar_df = spark.read.option("header", "true") \
    .csv(target_files_cal)

#  2. Transform 
df_calendar = calendar_df.select(
    col("listing_id").cast("long"),
    
    # Parse Date: yyyy-MM-dd
    to_date(col("date"), "yyyy-MM-dd").alias("calendar_date"),
    
    # Clean Boolean (t/f -> True/False)
    (when(col("available") == 't', True)
     .when(col("available") == 'f', False)
     .otherwise(None)).alias("is_available"),
     
    # Clean Price
    (regexp_replace(col("price"), "[$,]", "").cast("decimal(10, 2)")).alias("price"),
    
    col("minimum_nights").cast("int").alias("minimum_nights_on_date"),
    col("maximum_nights").cast("int").alias("maximum_nights_on_date")
).na.drop(subset=["listing_id", "calendar_date"])

# --- 3. Load ---
print("Writing Calendar data (this may take a moment)...")
df_calendar.write.mode("overwrite").parquet(os.path.join(PROCESSED_PATH, "calendar"))

print("Calendar Processed Successfully!")

Searching for: C:\Users\jinna\Downloads\inside-airbnb-bangkok-data\raw_data\*\calendar*.csv.gz
Found 4 files.
Writing Calendar data (this may take a moment)...
Calendar Processed Successfully!


In [5]:
# Create connection (file will be created at database/airbnb_dw.duckdb)
os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)
con = duckdb.connect(DB_PATH)

print("Loading data into DuckDB tables...")

# Create Tables from Parquet files
con.execute(f"""
    CREATE OR REPLACE TABLE dim_hosts AS 
    SELECT * FROM read_parquet('{os.path.join(PROCESSED_PATH, "hosts", "*.parquet")}');
""")

con.execute(f"""
    CREATE OR REPLACE TABLE dim_listings AS 
    SELECT * FROM read_parquet('{os.path.join(PROCESSED_PATH, "listings", "*.parquet")}');
""")

con.execute(f"""
    CREATE OR REPLACE TABLE fact_calendar AS 
    SELECT * FROM read_parquet('{os.path.join(PROCESSED_PATH, "calendar", "*.parquet")}');
""")

# Verify Results
print("--- Data Warehouse Statistics ---")
print("Hosts count:", con.execute("SELECT COUNT(*) FROM dim_hosts").fetchone()[0])
print("Listings count:", con.execute("SELECT COUNT(*) FROM dim_listings").fetchone()[0])
print("Calendar count:", con.execute("SELECT COUNT(*) FROM fact_calendar").fetchone()[0])

con.close()
print("Batch Pipeline Completed Successfully!")

Loading data into DuckDB tables...
--- Data Warehouse Statistics ---
Hosts count: 10023
Listings count: 35199
Calendar count: 40207699
Batch Pipeline Completed Successfully!


In [7]:
import duckdb
import pandas as pd
import os

BASE_PATH = r"C:\Users\jinna\Downloads\inside-airbnb-bangkok-data"
DB_PATH = os.path.join(BASE_PATH, "database", "airbnb_dw.duckdb")

con = duckdb.connect(DB_PATH)

def show_head_tail(table_name, order_col):
    print(f"\n{'='*20} Table: {table_name} {'='*20}")
    
    print(f"--- Head (First 5 rows) ---")
    df_head = con.execute(f"""
        SELECT * FROM {table_name} 
        ORDER BY {order_col} ASC 
        LIMIT 5
    """).df()
    display(df_head)
    
    # Tail (5 แถวสุดท้าย)
    print(f"--- Tail (Last 5 rows) ---")
    df_tail = con.execute(f"""
        SELECT * FROM {table_name} 
        ORDER BY {order_col} DESC 
        LIMIT 5
    """).df()
    display(df_tail)

show_head_tail("dim_hosts", "host_id")
show_head_tail("dim_listings", "listing_id")
show_head_tail("fact_calendar", "calendar_date") # หรือใช้ listing_id คู่กับ calendar_date ก็ได้

# Join (Listings + Hosts + Calendar)
print(f"\n{'='*20} Joined Data (Preview) {'='*20}")
print("--- Joining: Fact Calendar + Dim Listings + Dim Hosts ---")

df_joined = con.execute("""
    SELECT 
        h.host_id,
        h.host_name,
        l.listing_id,
        l.listing_name,
        l.neighbourhood,
        c.calendar_date,
        c.price as daily_price,
        c.is_available
    FROM fact_calendar c
    JOIN dim_listings l ON c.listing_id = l.listing_id
    JOIN dim_hosts h ON l.host_id = h.host_id
    WHERE c.is_available = true 
    ORDER BY c.calendar_date ASC, l.listing_id ASC
    LIMIT 10
""").df()

display(df_joined)

# 5. ปิดการเชื่อมต่อ
con.close()


==================== Table: dim_hosts ====================
--- Head (First 5 rows) ---


,host_id,host_name,host_since_date,host_location,host_response_time,host_response_rate,host_acceptance_rate,is_superhost,host_listings_count,is_identity_verified
0,21447,Teerapat,2009-06-14,"Krung Thep Maha Nakhon, Thailand",N/A,NaN,1.00,False,1,True
1,58920,Chindawan,2009-12-01,None,within a few hours,1.0,0.82,False,1,True
2,70413,Patiyuth,2010-01-13,"Bangkok, Thailand",within a few hours,1.0,NaN,False,3,True
3,108793,Eddie,2010-04-15,"Bangkok, Thailand",within an hour,1.0,1.00,False,4,True
4,120437,Nuttee,2010-05-08,"Bangkok, Thailand",N/A,NaN,0.67,False,1,True


--- Tail (Last 5 rows) ---


,host_id,host_name,host_since_date,host_location,host_response_time,host_response_rate,host_acceptance_rate,is_superhost,host_listings_count,is_identity_verified
0,720639377,Keerati,2025-09-24,None,N/A,NaN,NaN,False,1,True
1,720329562,荣生,2025-09-22,"Bangkok, Thailand",N/A,NaN,NaN,False,12,True
2,720111108,Anthiya,2025-09-21,None,N/A,NaN,NaN,False,1,True
3,719642511,Tum,2025-09-18,None,N/A,NaN,1.0,False,1,True
4,719481565,Lyna,2025-09-17,None,N/A,NaN,NaN,False,1,True



==================== Table: dim_listings ====================
--- Head (First 5 rows) ---


...

,listing_id,calendar_date,is_available,price,minimum_nights_on_date,maximum_nights_on_date
0,6773889,2024-12-25,False,406.0,1,1125
1,6398846,2024-12-25,False,750.0,1,1125
2,7887684,2024-12-25,False,699.0,28,1125
3,7736302,2024-12-25,True,1200.0,1,31
4,7777642,2024-12-25,False,1500.0,1,1125


--- Tail (Last 5 rows) ---


,listing_id,calendar_date,is_available,price,minimum_nights_on_date,maximum_nights_on_date
0,28590460,2026-09-27,False,NaN,1,1125
1,28590474,2026-09-27,False,NaN,1,1125
2,28590488,2026-09-27,False,NaN,1,1125
3,28590513,2026-09-27,False,NaN,1,7
4,26434680,2026-09-27,False,NaN,1,30



==================== Joined Data (Preview) ====================
--- Joining: Fact Calendar + Dim Listings + Dim Hosts ---


,host_id,host_name,listing_id,listing_name,neighbourhood,calendar_date,daily_price,is_available
0,703944,None,145343,Boutique Rooms Near Bangkok Airport,Lat Krabang,2024-12-25,1200.0,True
1,4115838,Primrose,873841,Suite,Thawi Watthana,2024-12-25,2905.0,True
2,4115838,Primrose,1035351,Room for Group stay 4pax,Thawi Watthana,2024-12-25,3500.0,True
3,4115838,Primrose,1035589,Double Bed Apartment,Thawi Watthana,2024-12-25,2000.0,True
4,4115838,Primrose,1035640,Double Bed Apartment,Thawi Watthana,2024-12-25,2000.0,True
5,26205111,Sethawat,5074762,Ban Kru Ae Homestay Don Muang DMK private room,Don Mueang,2024-12-25,1000.0,True
6,33894527,Wasin,6485868,FloralShire Suvarnabhumi Airport,Lat Krabang,2024-12-25,600.0,True
7,40660187,Phakakul,7736302,Little Mango lovely Home (with Love & Breakfast),Khlong Sam Wa,2024-12-25,1200.0,True
8,37135412,Thanakrit,10179988,บ้านเดี่ยว 2 ชั้น มีโรงจอดรถ 4 ห้องนอน 4 ห้องน้ำ,Taling Chan,2024-12-25,3200.0,True
9,26575230,K.,10935422,Kedkanok apartment,Sai Mai,2024-12-25,500.0,True
